In [1]:
import json
import requests
import pandas as pd

In [9]:
from elasticsearch8 import Elasticsearch
import json
from datetime import datetime
def config(k):
    with open(f'/configs/default/shared-data/{k}', 'r') as f:
        return f.read()

def main():
    try:
        es = Elasticsearch (
                'https://127.0.0.1:9200',
                request_timeout=60,
                verify_certs= False,
                ssl_show_warn= False,
                basic_auth=('elastic','elastic')
            )

        query = {
        "size":100,
            "_source": ["_id","phn_name","phn_code","dths_rspt3"],
            "query": {
                "match_all": {}
            }
        }


        result = es.search(index="premature_mortality", body=query)
        docs = result['hits']['hits']
        features = {}
        for doc in docs:
            _id = doc['_id']
            death_ratio = doc['_source']['dths_rspt3']/100000
            phn_name = doc['_source']['phn_name']
            phn_code = doc['_source']['phn_code']
            features[_id] = {
                '_id': _id,
                'death_ratio':death_ratio,
                'phn_code': phn_code,
                'phn_name': phn_name
            }

        ids = list(features.keys())
        
        aggregation_data = []
        for id in ids:
            print("searching: ", id)
            query = {
                "size": 10000,
                    "query": {
                        "bool": {
                            "must": [
                                {
                                    "exists": {
                                        "field": "geometry"
                                    }
                                }
                            ],
                            "filter": {
                                "geo_shape": {
                                    "geometry": {
                                        "indexed_shape": {
                                            "index": "premature_mortality",
                                            "id": id,
                                            "path": "geometry"
                                        }
                                    }
                                }
                            }
                        }
                    }
                }

            result2 = es.search(index="air-qualities", body=query)
            aggs_data = result2['hits']['hits']
            for data in aggs_data:
                if data['_source']['averageValue']!=None:
                    aggregation_data.append({
                        '_id': id,
                        'dataName': data['_source']['dataName'],
                        'avgValue': data['_source']['averageValue'],
                        'since': data['_source']['since']
                    })


        unique_data_names = set()
        unique_since = set()
        for data in aggregation_data:
            unique_data_names.add(data['dataName'])
            unique_since.add(data['since'])
        
        print(unique_data_names)
        sorted_date_times = sorted(unique_since, key=lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))

        print(len(unique_since))
        print(sorted_date_times)
        for id in ids:
            for data_name in unique_data_names:
                values = []
                for since in sorted_date_times:
                    values_since = []
                    for data in aggregation_data:
                        if data['_id'] == id and data['dataName'] == data_name and data['since']==since:
                            values_since.append(data['avgValue'])
                    if len(values_since)!=0:
                        values.append(sum(values_since)/len(values_since))
                    else:
                        values.append(None)
                features[id][data_name] = values
                
        response = {
            'status_code': 200,
            'data': list(features.values())
        }
        return json.dumps(response)
    except Exception as e:
        error_response = {'status_code': 500, 'error_message': str(e)}
        return json.dumps(error_response)
    

In [10]:
# response = requests.get("http://127.0.0.1:9090/EPA/air-quality-phn")
response = main()
data = json.loads(response).get('data')
data

searching:  HDczdY8B6E3HLROlX8Pq
searching:  GTczdY8B6E3HLROlN8Mz
searching:  HTczdY8B6E3HLROlacO3
searching:  ITczdY8B6E3HLROlhsOQ
searching:  HzczdY8B6E3HLROldsPG
searching:  IzczdY8B6E3HLROlk8ML
searching:  FzczdY8B6E3HLROlMsMR
searching:  GDczdY8B6E3HLROlMsMR
searching:  HjczdY8B6E3HLROlccOC
searching:  IDczdY8B6E3HLROlfMPa
searching:  GjczdY8B6E3HLROlQMM8
searching:  GzczdY8B6E3HLROlTMO5
searching:  IjczdY8B6E3HLROljMOu
searching:  JDczdY8B6E3HLROltcMf
searching:  JTczdY8B6E3HLROlucMS
searching:  JjczdY8B6E3HLROlu8NQ
searching:  JzczdY8B6E3HLROlvsPC
searching:  KDczdY8B6E3HLROlxMN_
searching:  KTczdY8B6E3HLROlz8Pw
searching:  KjczdY8B6E3HLROl38Mp
searching:  Kzc0dY8B6E3HLROlGsOk
searching:  LDc0dY8B6E3HLROlIsPM
searching:  LTc0dY8B6E3HLROlIsPM
searching:  Ljc0dY8B6E3HLROlIsPM
searching:  Lzc0dY8B6E3HLROlKcMK
searching:  MDc0dY8B6E3HLROlLcOd
searching:  MTc0dY8B6E3HLROlPMNi
searching:  Mjc0dY8B6E3HLROlU8MH
searching:  Mzc0dY8B6E3HLROlYcNN
searching:  NDc0dY8B6E3HLROlcsOi
searching:

[{'_id': 'HDczdY8B6E3HLROlX8Pq',
  'death_ratio': 0.000193111747522051,
  'phn_code': 'PHN205',
  'PHNName': 'Murray',
  'PM2.5': [2.0166666666666666,
   2.4933333333333336,
   4.378333333333333,
   8.0,
   9.843333333333334,
   6.456666666666666,
   6.488333333333333,
   5.498333333333332,
   5.408333333333332,
   3.01,
   3.7983333333333333,
   3.0300000000000007,
   3.4966666666666666,
   2.5016666666666665,
   8.195,
   3.8349999999999995,
   2.0616666666666665,
   1.3966666666666667,
   1.1116666666666668,
   0.6966666666666667,
   0.5366666666666666,
   0.7116666666666666,
   0.7916666666666666,
   0.8516666666666666,
   3.1516666666666673,
   8.543333333333335,
   9.25,
   6.776666666666667,
   8.338333333333331,
   8.081666666666665,
   8.071666666666667,
   4.645,
   4.863333333333333,
   4.034999999999999,
   4.538333333333334,
   5.196666666666666,
   4.905,
   5.421666666666667,
   5.546666666666667,
   4.798333333333334,
   4.515000000000001,
   3.356666666666667,
   3.248

In [31]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
df = pd.json_normalize(data)  # 使用 json_normalize() 函数加载数据到 DataFrame


# ['Particles', 'PM2.5', 'PM10', 'Visibility', 'API', 'O3', 'NO2', 'SO2', 'CO']
# 假设 df 是你的 DataFrame，features 是要展开的特征列表
features = ['Particles', 'PM2.5', 'PM10', 'Visibility', 'API', 'O3', 'NO2', 'SO2', 'CO']

# # 对于每个特征，将列表展开成新的列
# for feature in features:
#     # 对于每个特征，将列表展开成新的列，并命名为原特征名加上索引
#     df = pd.concat([df, df[feature].apply(pd.Series).add_prefix(feature+'_')], axis=1)
# 筛选出只包含数值类型的列
numeric_df = df[features].select_dtypes(include=[np.number])

# 计算数值列的平均值，同时将 None 替换为 NaN
mean_values = numeric_df.replace({None: np.nan}).mean()
# 将 None 替换为相应特征的平均值
for feature in features:
    df[feature] = df[feature].apply(lambda x: mean_values[feature] if x is None else x)

# 删除原始的特征列
df.drop(columns=features, inplace=True)

df = df.dropna(subset=df.columns[4:], how='all')

df = df.drop(columns=['_id','phn_code', 'PHNName'])
correlation_matrix = df.corr()
# 绘制热图
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix[['death_ratio']], annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix with death_rate")
plt.show()



TypeError: Could not convert [list([5.506875, 5.45, 6.251250000000001, 6.124999999999999, 8.201249999999998, 8.828125000000002, 8.769333333333332, 7.720625000000001, 7.838750000000001, 7.935333333333333, 8.45375, 8.530666666666665, 8.485000000000001, 8.45625, 9.660625000000003, 7.6674999999999995, 5.349375, 4.716249999999999, 4.492500000000001, 4.2681249999999995, 4.2325, 4.200625, 4.30875, None, None, 8.57, 10.355625, 11.13875, 11.944375000000003, 10.78125, 11.565, 10.056875000000002, 8.894374999999998, 8.680625, 10.246875000000003, 9.445625000000001, 10.258, 9.646875000000001, 14.132500000000002, 10.34, 7.845000000000001, 6.655625000000001, 5.78, 5.423749999999998, 5.356250000000001, 5.221999999999999, 5.819, 5.390000000000001, 5.700476190476191, 5.9228571428571435, 6.297142857142857, 6.733809523809525, 7.1034999999999995, 7.371904761904762, 6.506666666666666, 6.142380952380954, 5.977500000000001, 5.981904761904762, 6.094761904761905, 6.149523809523809, 7.098571428571427, 6.051904761